In [1]:
using Revise
using DataStructures
using Distributions
using Quarton

### Example: Network of queues with non-probabilistic routing

Page 17. This represents disk usage for a job.

 1. Jobs arrive at an exponential rate.
 2. A CPU handles those jobs and feeds them to the queue for Disk 1.
 3. Disk 1 processes a job and either sends it to the sink or to Disk 2.
    a. If the job has a mark of 1, it goes to Disk 2 and gets incremented.
    b. If the job has a mark of 2, it goes to the sink.
 4. Disk 2 processes jobs and returns them to the queue of Disk 1.


In [7]:
function nonprobabilistic_routing()
    arrival_rate = 1.0
    service_rate = 1.0
    T = CountedWork
    model = QueueModel{T}()
    token_builder = (when, rng) -> T(1.0, when)
    source = InfiniteSourceQueue{T}(token_builder)
    arrival = ArrivalServer(arrival_rate)
    @pipe! model source => arrival :only
    CPU_queue = FIFOQueue{T}()
    @pipe! model arrival => CPU_queue :only
    CPU = ModifyServer(service_rate)
    @pipe! model CPU_queue => CPU :only

    disk1_queue = FIFOQueue{T}()
    @pipe! model CPU => disk1_queue :only

    on_output = token -> (token.mark = 2; nothing)
    assign_by_mark = SizeIntervalAssignment(t -> (t.mark == 1) ? :around : :out)
    disk1 = ModifyServer(
        service_rate, disbursement=assign_by_mark, modify=on_output
        )
    @pipe! model disk1_queue => disk1 :only

    sink = SummarySink{T}()
    @pipe! model disk1 => sink :out
    
    disk2_queue = FIFOQueue{T}()
    @pipe! model disk1 => disk2_queue :around

    disk2 = ModifyServer(service_rate)
    @pipe! model disk2_queue => disk2 :only
    @pipe! model disk2 => disk1_queue :only
    check_model(model)
    println("servers $(model.s_name)")
    println("queues $(model.q_name)")

    trajectory = Trajectory(2342334)
    start_time = zero(Float64)
    activate!(model, trajectory, arrival, T())
    when = 0.0
    cnt = 0
    while when < 100.0 && cnt < 10000
        when, which = next(trajectory)
        step!(model, trajectory, (when, which))
        cnt += 1
    end
    return throughput(sink)
end
nonprobabilistic_routing()

servers Dict("disk1" => 3, "CPU" => 2, "arrival" => 1, "disk2" => 4)
queues Dict("sink" => 4, "CPU_queue" => 2, "disk1_queue" => 3, "source" => 1, "disk2_queue" => 5)


┌ Debug: Bulding queueing model
└ @ Quarton /home/adolgert/dev/quarton/src/model.jl:133


0.025293862377925223

### Example: Finite buffer

This buffer will drop jobs when it is full. This is on page 17.

In [8]:
function finite_buffer(arrival_rate, service_rate, keep_cnt)
    T = Work
    model = QueueModel{T}()
    source = InfiniteSourceQueue{T}()
    fifo = FiniteFIFOQueue{T}(keep_cnt)
    sink = SummarySink{T}()
    s1 = ArrivalServer(arrival_rate)
    s2 = ModifyServer(service_rate)
    @pipe! model source => s1 :only
    @pipe! model s1 => fifo :only
    @pipe! model fifo => s2 :only
    @pipe! model s2 => sink :only
    check_model(model)
    trajectory = Trajectory(2342334)
    start_time = zero(Float64)
    activate!(model, trajectory, s1, Work())
    when = 0.0
    while when < 10000.0
        when, which = next(trajectory)
        step!(model, trajectory, (when, which))
    end
    response = sink.retire_total_duration / sink.retire_cnt
    drops = fifo.drop_cnt
    return response, drops
end
# How many drops do we get when the rate of arrivals equals the rate
# of the server, over the 10,000s?
keep_cnt = 9
finite_buffer(1.0, 1.0, keep_cnt)

┌ Debug: Bulding queueing model
└ @ Quarton /home/adolgert/dev/quarton/src/model.jl:133


(6.037175250004293, 943)